<a href="https://colab.research.google.com/github/cu2uyash/Spreadsheet-Parser/blob/master/imagerecognitiontraining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from pathlib import Path

Using TensorFlow backend.


In [2]:
# Loading the data  set # it returns 4 parameters the training and testing data with coressponding labels
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

170500096/170498071 [==============================] - 2s 0us/step


In [ ]:
#NN works best when input data are floating points are  values and are b/w 0 and 1 normally images are stored in form of pixels values b/w 0-255 so we convert it from integer to floating point values
# Normalize data set to 0-to-1 range
x_train =x_train.astype("float32")
x_test =x_test.astype("float32")
#now we scale the data so its between 0 and 1 that can be done by simply diving the data by max value that is 255 So this way data is normalized.
# as x_train and x_test are numpy array it will divide every value of array with 255(For more detail refer to the concept of board-casting.)
x_train =x_train/255
x_test =x_test/255

# Convert class vectors to binary class matrices
# Our labels are single values from 0 to 9.
# Instead, we want each label to be an array with on element set to 1 and and the rest set to 0.
y_train =keras.utils.to_categorical(y_train, 10)
y_test =keras.utils.to_categorical(y_test, 10)
# Create a model and add layers
#we will create a squential model it is called squential because we add layers in sequence and it automatically gets connected set by step
model =Sequential()
#we will add a convolutional layers as it work best on images for detection of various king of patterns in a image thr are tow types of convolutional layers 1D and 2D since we are working with images we will use 2D layer
#the first parameter is how many diff filter each will be capable to detect one pattern in image next is size of window that will be used to get images tiles form the image thsi will split up the image in 3*3 window size while doing this if the window size is divisible of image size then fine or to get it done we can add padding to the image that is adding extra zeros to the image.
#next is the activation function and since its the first layer we need to specify the input dimensions of image
model.add(Conv2D(32,(3,3),padding="same",activation='relu',input_shape=(32,32,3)))
#to make model more efficient we will add more such convolutional layers
model.add(Conv2D(32,(3,3),activation='relu'))
#Max pooling is to scale down the o/p of Convolutional layers with higher values and throwing away the smaller ones this makes NN more efficient
#The parameter we need to specify is the pool size that is it will make a matrix of 2*2 and select the largest value from them. keeping the matrix small with imp values
model.add(MaxPooling2D(pool_size=(2,2)))
#One problem with NN is they tend to memorise data instead of learning so to eliminate this we will add a dropout layer
#we will randomly hrow up some data by cutting down some connection b/w layers
#the only parameter we need to add is percntg of connection need to be cut
model.add(Dropout(0.25))
model.add(Conv2D(64,(3,3),padding='same',activation='relu'))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

#whenever we make tramsitional between a conv2D layer and dense layer we need to tell NN that we no longer working with 2D data for that we use flatten function
model.add(Flatten())
#In the below line of code we are adding a Dense layer with  512 node and WE are specifying a activation function
model.add(Dense(512,activation='relu'))
model.add(Dropout(0.50))
#for the output since we 10 diff objects so we will take 10 nodes and since the o/p will be of one object per image we will use softmax as a activation function
model.add(Dense(10,activation='softmax'))

#As the model is built by adding diff layers the final step is to compile the model
#here we need to specify  the loss fuction that is how much right or wrong your training is next we specify the optimizer
model.compile(loss='categorical_crossentropy',optimizer='adam')
# TO see how models looks like
# Print a summary of the model
model.summary()
#train the model
#to train the model we need to call model.fit()
# we need to specify the training dataset and corresponding labels
#next we need to pass batch size that is how many images we need to fit in during training set the number not too low not too high
#then we need to specify number of epoch that is one pass through the total dataset
#next we give the validation data that network has never seen
#and to randomise data we set shuffle.
model.fit(
    x_train,
    y_train,
    batch_size=64,
    epochs=30,
    validation_data=(x_test, y_test),
    shuffle=True
)

# Save neural network structure
model_structure = model.to_json()
f = Path("model_structure.json")
f.write_text(model_structure)

# Save neural network's trained weights
model.save_weights("model_weights.h5")

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 64)        18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 13, 13, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 6, 64)         